#### YOLOv8 Training

In [ ]:
!pip install ultralytics
!pip install roboflow

In [ ]:
import ultralytics
from ultralytics import YOLO
from IPython.display import Image
from roboflow import Roboflow

In [ ]:

rf = Roboflow(api_key="your api key")
project = rf.workspace("fight-detection").project("fight-detection-new")
version = project.version(3)
dataset = version.download("yolov8")


In [ ]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/Fight-Detection-New-3/data.yaml batch=-1 epochs=100 save=True imgsz=640 patience=50 plots=True weight_decay=0.001


In [ ]:
!yolo val model=/content/runs/detect/train/weights/best.pt data=/content/Fight-Detection-New-3/data.yaml split=test


#### YOLOv8 Implementation

In [ ]:
import ultralytics
from ultralytics import YOLO
from IPython.display import Image

In [ ]:
model = YOLO('fight_det_model.pt')
# path of the model

In [ ]:
model.predict(source="source vide path", imgsz=640, conf=0.5, save=True, device=0)
# source= video/image path


#### YOLO-NAS Training

In [ ]:
!pip install -q git+https://github.com/Deci-AI/super-gradients.git@stable
!pip install -q roboflow
!pip install -q supervision

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
MODEL_ARCH = 'yolo_nas_m'

In [ ]:
from super_gradients.training import models

model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)

In [ ]:
%cd {HOME}
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="your api key")
project = rf.workspace("fight-detection").project("fight-detection-new")
version = project.version(11)
dataset = version.download("yolov5")


In [ ]:
LOCATION = dataset.location
print("location:", LOCATION)
CLASSES = sorted(project.classes.keys())
print("classes:", CLASSES)

In [ ]:
MODEL_ARCH = 'yolo_nas_m'
BATCH_SIZE = 8
MAX_EPOCHS = 100
CHECKPOINT_DIR = f'{HOME}/checkpoints'
EXPERIMENT_NAME = project.name.lower().replace(" ", "_")

In [ ]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': CLASSES
}

In [ ]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

In [ ]:
train_data.dataset.transforms

In [ ]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)

In [ ]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [ ]:
trainer.train(
    model=model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)

In [ ]:
!zip -r yolo_nas.zip {CHECKPOINT_DIR}/{EXPERIMENT_NAME}

In [ ]:
avg_model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    checkpoint_path="average_model.pth"
).to(DEVICE)

In [ ]:
trainer.test(
    model=avg_model,
    test_loader=test_data,
    test_metrics_list=DetectionMetrics_050(
        score_thres=0.1,
        top_k_predictions=300,
        num_cls=len(dataset_params['classes']),
        normalize_targets=True,
        post_prediction_callback=PPYoloEPostPredictionCallback(
            score_threshold=0.01,
            nms_top_k=1000,
            max_predictions=300,
            nms_threshold=0.7
        )
    )
)

In [ ]:
best_model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    checkpoint_path="ckpt_best.pth"
).to(DEVICE)

In [ ]:
trainer.test(
    model=best_model,
    test_loader=test_data,
    test_metrics_list=DetectionMetrics_050(
        score_thres=0.1,
        top_k_predictions=300,
        num_cls=len(dataset_params['classes']),
        normalize_targets=True,
        post_prediction_callback=PPYoloEPostPredictionCallback(
            score_threshold=0.01,
            nms_top_k=1000,
            max_predictions=300,
            nms_threshold=0.7
        )
    )
)

#### YOLO-NAS Implementation

In [ ]:
!pip install super_gradients
!pip install imutils
!pip install pytube --upgrade

In [ ]:
from super_gradients.training import models
model = models.get('yolo_nas_m', 
                   num_classes=2, 
                   pretrained_weights="coco"
                   )

In [ ]:
best_model = models.get('yolo_nas_m',
                        num_classes=2,
                        checkpoint_path="average_model.pth") # model path

In [ ]:
import torch
device = 0 if torch.cuda.is_available() else "cpu"

input_video_path = "source video path"
output_video_path = "output video path"
#device=0

best_model.to(device).predict(input_video_path, conf=0.01).save(output_video_path)

#### YOLOv10 Training

In [ ]:
!git clone https://github.com/THU-MIG/yolov10.git

In [ ]:
%cd yolov10

In [ ]:
import os 
os.getcwd()

In [ ]:
pip install .

In [ ]:
import os
import urllib.request

# Create a directory for the weights in the current working directory
weights_dir = os.path.join(os.getcwd(), "weights")
os.makedirs(weights_dir, exist_ok=True)

# URLs of the weight files
urls = [
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10s.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10m.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10b.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10x.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10l.pt"
]

# Download each file
for url in urls:
    file_name = os.path.join(weights_dir, os.path.basename(url))
    urllib.request.urlretrieve(url, file_name)
    print(f"Downloaded {file_name}")

In [ ]:
%cd /workspace/datasets/datasets/datasets

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="your api key")
project = rf.workspace("fight-detection").project("fight-detection-new")
version = project.version(11)
dataset = version.download("yolov8")

In [ ]:
!pip install huggingface_hub

In [ ]:
!yolo task=detect mode=train epochs=100 batch=-1 plots=True save=True patience=50 model=/workspace/yolov10/weights/yolov10b.pt data=Fight-Detection-New-11/data.yaml 
#

In [ ]:
!yolo val model=/workspace/datasets/datasets/datasets/runs/detect/train/weights/best.pt data=Fight-Detection-New-11/data.yaml split=test


In [ ]:
!yolo task=detect mode=predict conf=0.25 save=True show=True model=/workspace/datasets/datasets/datasets/runs/detect/train/weights/best.pt source=21.avi
#

#### YOLOv10 Implementation

In [ ]:
!git clone https://github.com/THU-MIG/yolov10.git

In [ ]:
%cd yolov10

In [ ]:
!pip install .

In [ ]:
!yolo task=detect mode=predict conf=0.25 save=True model='model path' source='source video path'